In [1]:
D.<x> = PolynomialRing(QQ)
R.<x> = PolynomialRing(QQ)
S = PowerSeriesRing(QQ,'x', default_prec=1000)
L=LaurentSeriesRing(QQ,'x', default_prec=1000)
#S.<x,y>=PowerSeriesRing(QQ,default_prec=1000)
# Using "R = PolynomialRing(QQ,'x')" produces an error.
# Yet using S as defined here is fine (see definition of Fraleigh4 below)
# But S = PowerSeriesRing(QQ,'x','y') also produces an error.

def uniteList(lst):
    ln=len(lst)
    if ln<2:return lst
    answer=[]
    for j in [0..ln-1]:
        lj=lst[j]
        if (lj in answer)==False:
            answer=answer+[lj]
    return answer

def delete_element(elt,ls):
    new=[]
    for j in ls:
        if j!=elt:new=new+[j]
    return new

def sortAndSmash(lst): #sorts and deletes repeats
    cropped=lst
    new=[]
    while len(cropped)>0:
        m=min(cropped)
        new=new+[m]
        cropped=delete_element(m,cropped)
    return(new)

def difference_table(lst):
    diffs=[]
    for k in [0..len(lst)-2]:
        diff=lst[k+1]-lst[k]
        diffs=diffs+[diff]
    return diffs

def evaluate(expression,incomingvalue):
    expr=str(expression)
    output=sage_eval(expr,locals={'x':incomingvalue})
    return output

P=Primes()

def prime(n): return P.unrank(n-1)

def unite(A,B):
    return set(A).union(B)

def factor_integer(n): #this is like "factor(integer)", but
    #the bases and exponents can be easily extracted.
    F=factor(n)
    return list(F)

def orderInteger(p,n):
    if n!=0:
        lst=n.digits(p)
        count=0
        k=0
        while lst[k]==0:
            count=count+1
            k=k+1
        return count
    if n==0:return(Infinity)

def order(p,fr):
    nn=numerator(QQ(fr))
    dn=denominator(QQ(fr))
    return orderInteger(p,nn)-orderInteger(p,dn)

def digitsum(p,n):return add(n.digits(p))

def digitsum(p,n):
    return sum(n.digits(p))

def rmnjntau(n):
    answer=0
    for k in [1..n-1]:
        answer=answer+(6*k-4*(n-k))*sigma(k,3)*sigma(n-k,5)
    answer=(5*sigma(n,3)+7*sigma(n,5))*n/12-35*answer
    return answer

def pochhammer(x,n):
    return product ([x+k for k in [0..n-1]])

def cRaleigh2(alpha,beta,nu):
    pchhmrA = pochhammer(alpha,nu)
    pchhmrB = pochhammer(beta,nu)
    fnu=factorial(nu)
    answer = D(pchhmrA*pchhmrB/fnu^2)
    return answer

def term(alpha,beta,p):
    return D(1/(alpha + p) + 1/(beta + p) - 2/(1 + p))

def eRaleigh(alpha, beta, nu):
    return D(sum([term(alpha, beta, p) for p in [0..nu - 1]]))

def FstarRaleigh2(alpha,beta,u,terms):
    fsr = sum([cRaleigh2(alpha, beta, nu)*eRaleigh(alpha,beta,nu)*u^nu for nu in [1..terms]])
    return D(fsr)

def Fraleigh2(alpha,beta,u,terms): 
    return D(sum([cRaleigh2(alpha,beta,nu)*u^nu for nu in [0..terms]]))

def FstarRaleigh3(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fsr2 = FstarRaleigh2(alpha,beta,x,n)
    return D(fsr2)

def Fraleigh4(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fr2 = Fraleigh2(alpha,beta,x,n)
    return D(fr2)

def exNo3c(n,m):
    a1 = S(x)*exp(S(FstarRaleigh3(n,m,x)/Fraleigh4(n,m,x))).O(2*n)
    # 
    # S(x) instead of x in df of a1 to avoid an error message
    # after use of the stripQuotationMarks command below.
    a2 = S(a1).O(n+1)
    return a2 # applying S to argument of exp is key.

def J(n,m):
    exn = exNo3c(n+2,m)
    jnm = 1/exn.reverse()
    return jnm.O(n+1) # Jtest is faster than J, and they agree.
def normalJ(n,m):
    f = J(n,m)
    return f.truncate(n+1)

def Jcoefficient(n,m):
    f = J(n+2,m)
    f1 = f.truncate(n+1)
    f1=f1.list()
    f1=Set(f1)
    g1 = f.truncate(n)
    g1 = g1.list()
    g1 = set(g1)
    h = f1.difference(g1)
    h = h.list()
    return h[0]

def xjNew(n,m):
    def xjNewStep(n,m):
        f=J(n,m)
        fl=f.list()
        rt=0
        for i in [0..len(fl)-1]:
            rt=rt+fl[i]*x**i*(m**3*2**6)**i
        return rt
    return xjNewStep(n-1,m)

def j(n,m):return L(xjNew(n,m)/x)

def xjLeoNew(n,m,multiplier):
    def xjNewStep(n,m):
        f=J(n,m)
        fl=f.list()
        rt=0
        for i in [0..len(fl)-1]:
            rt=rt+fl[i]*x**i*(multiplier)**i
        return rt
    return xjNewStep(n-1,m)

def jLeo(n,m,multiplier):return L(xjLeoNew(n,m,multiplier)/x)

def JpowerConstantTerm(m,power):
    trun=J(power,m)
    trun=expand(trun^power)
    polyversion=expand(trun*x^power)
    constant=polynomialCoefficient(power,polyversion)
    return constant

def jpowerConstantTerm(m,power):
    trun=j(power,m)
    trun=expand(trun^power)
    polyversion=expand(trun*x^power)
    constant=polynomialCoefficient(power,polyversion)
    return constant

def polynomialCoefficient(n,poly):
    return poly.list()[n]

def leadingCoefficient(polynomial):
    dg=polynomial.degree()
    
    return polynomialCoefficient(dg,polynomial)

def factorpairToFactor(factorpair):
    (monomial,exponent)=factorpair
    return monomial^exponent

def numericalfactor(polynomial):
    fp=polynomial.factor()
    lst=[factorpairToFactor(factorpair) for factorpair in fp]
    return polynomial/product(lst)

def firstfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.first())

def lastfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.last())

def reduced(polynomial):
    return numericalfactor(polynomial)*lastfactor(polynomial)

def element(k,listo):
    if k>len(listo):return (print("bad k"))
    return listo[k-1]

def mobiusInverseOfFunction(g,n):
    dv=divisors(n)
    nd=len(dv)
    f=0
    for k in [1..nd]:
        dvk=element(k,dv)
        f=f+moebius(dvk)*g(n/dvk)
    return f

def mobiusInverseOfList(g,n):
    dv=divisors(n)
    nd=len(dv)
    f=0
    for k in [1..nd]:
        dvk=element(k,dv)
        gndvk=element(n/dvk,g)
        f=f+moebius(dvk)*gndvk
    return f

def drop_first_element(listo):
    lenth=len(listo)
    answer=[]
    for k in [1..lenth-1]:
        answer=answer+[listo[k]]
    return answer

def factorSeriesFromLists(lsst,bound):
    truncatedlist=drop_first_element(lsst)
    F=[element(1,truncatedlist)]
    nswr=[-mobiusInverseOfList(F,1)/1]
    for n in [2..bound]:
        summ=0
        for k in [1..n-1]:
            ekf=element(k,F)
            tnk=element(n-k,truncatedlist)
            summ=summ+ekf*tnk
        Fn=n*element(n,truncatedlist)-summ
        F=F+[Fn]
        xpn=-mobiusInverseOfList(F,n)/n
        nswr=nswr+[xpn]
    return nswr

def factorSeriesFromListsWithTracking(lsst,bound):
    truncatedlist=drop_first_element(lsst)
    F=[element(1,truncatedlist)]
    nswr=[-mobiusInverseOfList(F,1)/1]
    for n in [2..bound]:
        summ=0
        for k in [1..n-1]:
            ekf=element(k,F)
            tnk=element(n-k,truncatedlist)
            summ=summ+ekf*tnk
            print((n,k))
        Fn=n*element(n,truncatedlist)-summ
        F=F+[Fn]
        xpn=-mobiusInverseOfList(F,n)/n
        nswr=nswr+[xpn]
        print("answer length: ",len(nswr))
    return nswr

def EulerMultiplication(listo):
    # Lemma 2.11 from my 1998 Experimental Mathematics paper.
    # Here the set A in the lemma is {1, 2, ...} = Z^+.
    p_A=[1] # so p_A[0]=1 as on top of p. 266, column 2.
    def ex(n):return listo[n-1] # ex is the exponent on the factor 1-x^n: -f(n)/n.
    def f(n):return -n*ex(n)
    answer=[1]
    def f_A(k):
        summ=0
        dk=divisors(k)
        for d in dk: #Here is where A = Z^+ is used to drop a constraint on k.
            summ=summ+f(d)
        return summ
    for n in [1..len(listo)-1]:
        sm=0
        for k in [1..n]:
            sm=sm+f_A(k)*p_A[n-k]
        p_A=p_A+[sm/n]
    return p_A

def listToPoly(listo):
    lenth=len(listo)
    answer=0
    for k in [1..lenth]:
        cf=listo[k-1]
        answer=answer+cf*x^(k-1)
    return answer

def stripQuotationMarks(string):
    var('x')
    return sage_eval(string,locals={'x':x})

print("okay1")

okay1


In [8]:
for n in [1..5]:
    print((n,sigma(n,k=11)))

(1, 1)
(2, 2049)
(3, 177148)
(4, 4196353)
(5, 48828126)


In [6]:
import time

def p2RamanujanCongruenceSeries(n): # Ramanujan Revisted, p. 293
    ans=0
    for a in [1..n+1]:
        o2=order(2,a)
        purepart=2^o2
        oddpart=a/purepart
        d=sigma(oddpart,k=11)
        
        term=d*8^o2*x^a        
        ans=ans+term
    return D(ans)

def recipP2(n):
    ans=L(1/p2RamanujanCongruenceSeries(n))
    ans=ans.O(n+1)
    return ans

def p2RecipPowerConstantTerm(k):
    g=recipP2(k)^k
    g=g.O(k+1)
    #print("g:");print(g);print()
    #print("list:")
    gl=g.list()
    #print(gl);print()
    ln=len(gl)
    return gl[ln-2]
    

L=LaurentSeriesRing(QQ,'x', default_prec=1010)

tests=[]
start=time.time()
for a in [1..1000]: #default precision must be > max a.
    c=p2RecipPowerConstantTerm(a)
    o=order(3,c)
    ds=digitsum(3,a)
    finish=time.time()
    test=mod(c,3)
    tests=tests+[test]
    print((a,o,test))
print(tests)

(1, 0, 1)
(2, 0, 1)
(3, 0, 1)
(4, 0, 1)
(5, 0, 1)
(6, 0, 1)
(7, 0, 1)
(8, 0, 1)
(9, 0, 1)
(10, 0, 1)
(11, 0, 1)
(12, 0, 1)
(13, 0, 1)
(14, 0, 1)
(15, 0, 1)
(16, 0, 1)
(17, 0, 1)
(18, 0, 1)
(19, 0, 1)
(20, 0, 1)
(21, 0, 1)
(22, 0, 1)
(23, 0, 1)
(24, 0, 1)
(25, 0, 1)
(26, 0, 1)
(27, 0, 1)
(28, 0, 1)
(29, 0, 1)
(30, 0, 1)
(31, 0, 1)
(32, 0, 1)
(33, 0, 1)
(34, 0, 1)
(35, 0, 1)
(36, 0, 1)
(37, 0, 1)
(38, 0, 1)
(39, 0, 1)
(40, 0, 1)
(41, 0, 1)
(42, 0, 1)
(43, 0, 1)
(44, 0, 1)
(45, 0, 1)
(46, 0, 1)
(47, 0, 1)
(48, 0, 1)
(49, 0, 1)
(50, 0, 1)
(51, 0, 1)
(52, 0, 1)
(53, 0, 1)
(54, 0, 1)
(55, 0, 1)
(56, 0, 1)
(57, 0, 1)
(58, 0, 1)
(59, 0, 1)
(60, 0, 1)
(61, 0, 1)
(62, 0, 1)
(63, 0, 1)
(64, 0, 1)
(65, 0, 1)
(66, 0, 1)
(67, 0, 1)
(68, 0, 1)
(69, 0, 1)
(70, 0, 1)
(71, 0, 1)
(72, 0, 1)
(73, 0, 1)
(74, 0, 1)
(75, 0, 1)
(76, 0, 1)
(77, 0, 1)
(78, 0, 1)
(79, 0, 1)
(80, 0, 1)
(81, 0, 1)
(82, 0, 1)
(83, 0, 1)
(84, 0, 1)
(85, 0, 1)
(86, 0, 1)
(87, 0, 1)
(88, 0, 1)
(89, 0, 1)
(90, 0, 1)
(91, 0, 1)
(92, 0, 

(693, 0, 1)
(694, 0, 1)
(695, 0, 1)
(696, 0, 1)
(697, 0, 1)
(698, 0, 1)
(699, 0, 1)
(700, 0, 1)
(701, 0, 1)
(702, 0, 1)
(703, 0, 1)
(704, 0, 1)
(705, 0, 1)
(706, 0, 1)
(707, 0, 1)
(708, 0, 1)
(709, 0, 1)
(710, 0, 1)
(711, 0, 1)
(712, 0, 1)
(713, 0, 1)
(714, 0, 1)
(715, 0, 1)
(716, 0, 1)
(717, 0, 1)
(718, 0, 1)
(719, 0, 1)
(720, 0, 1)
(721, 0, 1)
(722, 0, 1)
(723, 0, 1)
(724, 0, 1)
(725, 0, 1)
(726, 0, 1)
(727, 0, 1)
(728, 0, 1)
(729, 0, 1)
(730, 0, 1)
(731, 0, 1)
(732, 0, 1)
(733, 0, 1)
(734, 0, 1)
(735, 0, 1)
(736, 0, 1)
(737, 0, 1)
(738, 0, 1)
(739, 0, 1)
(740, 0, 1)
(741, 0, 1)
(742, 0, 1)
(743, 0, 1)
(744, 0, 1)
(745, 0, 1)
(746, 0, 1)
(747, 0, 1)
(748, 0, 1)
(749, 0, 1)
(750, 0, 1)
(751, 0, 1)
(752, 0, 1)
(753, 0, 1)
(754, 0, 1)
(755, 0, 1)
(756, 0, 1)
(757, 0, 1)
(758, 0, 1)
(759, 0, 1)
(760, 0, 1)
(761, 0, 1)
(762, 0, 1)
(763, 0, 1)
(764, 0, 1)
(765, 0, 1)
(766, 0, 1)
(767, 0, 1)
(768, 0, 1)
(769, 0, 1)
(770, 0, 1)
(771, 0, 1)
(772, 0, 1)
(773, 0, 1)
(774, 0, 1)
(775, 0, 1)
(776